In [1]:
%pip install ipykernel -U --user --force-reinstall

ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install --upgrade pip setuptools wheel
%pip install numpy
%pip install tensorflow==2.18.0
%pip install tflite-runtime==2.14.0
%pip install opencv-python==4.7.0.72
%pip install mediapipe

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.7.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached termcolor-3.2.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached wrapt-2.0.1-cp310-cp310-manylinux1_x86_64.manylinux_2_28_x86_64.manylinux_2_5_x86_64.whl.metadata (9.0 kB)
  Using cached grpcio-1.76.0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (3.7 kB)
  Using cached keras-3.12.0-py3-none-any.whl.meta

In [ ]:
import cv2

# Open the default camera (0 usuammlly refers to the first camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Failed to grab frame")
        break
    
    # Display the resulting frame
    cv2.imshow('Camera', frame)
    
    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()


In [2]:
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'Nothing', 'O', 'P', 'Q', 'R', 'S', 'Space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [ ]:
import cv2
import numpy as np
import mediapipe as mp

import tflite_runtime.interpreter as tflite

class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'Nothing', 'O', 'P', 'Q', 'R', 'S', 'Space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
TARGET_SIZE = 224
MODEL_PATH = "/home/rafayahmadraza/SignBuddy/Models/asl_model_1.tflite"   # your trained model file


# -------------------------------------------------------
# Resize the cropped hand to 224×224 while keeping aspect
# -------------------------------------------------------
def resize_with_padding(img, target=TARGET_SIZE):
    h, w = img.shape[:2]
    aspect = w / h

    if aspect > 1:  # wide
        new_w = target
        new_h = int(target / aspect)
    else:  # tall
        new_h = target
        new_w = int(target * aspect)

    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # Add black padding
    top = (target - new_h) // 2
    bottom = target - new_h - top
    left = (target - new_w) // 2
    right = target - new_w - left

    padded = cv2.copyMakeBorder(
        resized, top, bottom, left, right,
        cv2.BORDER_CONSTANT, value=(0, 0, 0)
    )
    return padded


# -------------------------------------------------------
# Load TFLite Model
# -------------------------------------------------------
interpreter = tflite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Model input:", input_details[0]["shape"])


# -------------------------------------------------------
# Initialize MediaPipe Hands
# -------------------------------------------------------
mp_draw = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

cap = cv2.VideoCapture(0)


# -------------------------------------------------------
# Main loop
# -------------------------------------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:

            # Extract bounding box
            xs = [lm.x for lm in hand_landmarks.landmark]
            ys = [lm.y for lm in hand_landmarks.landmark]

            xmin = int(min(xs) * w)
            xmax = int(max(xs) * w)
            ymin = int(min(ys) * h)
            ymax = int(max(ys) * h)

            pad = 30
            xmin = max(0, xmin - pad)
            ymin = max(0, ymin - pad)
            xmax = min(w, xmax + pad)
            ymax = min(h, ymax + pad)

            # Draw box
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

            # Crop
            crop = frame[ymin:ymax, xmin:xmax]

            if crop.size > 0:
                crop_96 = resize_with_padding(crop)

                # Show resized hand
                cv2.imshow("Hand 96x96", crop_96)

                # Prepare input for model
                inp = cv2.cvtColor(crop_96, cv2.COLOR_BGR2RGB)
                inp = inp.astype(np.float32) / 255.0
                inp = np.expand_dims(inp, axis=0)

                # Run inference
                interpreter.set_tensor(input_details[0]["index"], inp)
                interpreter.invoke()
                prediction = interpreter.get_tensor(output_details[0]["index"])

                # Get predicted class
                cls = np.argmax(prediction)
                conf = np.max(prediction)

                # Display prediction
                text = f"Class: {class_names[np.argmax(prediction)]}  ({conf:.2f})"
                cv2.putText(frame, text, (xmin, ymin - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                # Draw landmarks
                mp_draw.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS
                )

    cv2.imshow("Live", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Model input: [  1 224 224   3]


I0000 00:00:1765174136.521368    6513 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1765174136.523038    7212 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.1.5-1pop0~1756399231~22.04~b84bab8), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
W0000 00:00:1765174136.550751    7202 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765174136.564494    7208 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [10]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model

# -------------------------------------------------------
# SETTINGS
# -------------------------------------------------------
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
               'Nothing', 'O', 'P', 'Q', 'R', 'S', 'Space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

TARGET_SIZE = 96
MODEL_PATH = "/home/rafayahmadraza/SignBuddy/Models/asl_tiny_model_2.keras"   # 👈 now .h5 file


# -------------------------------------------------------
# Resize the cropped hand to 224×224 while keeping aspect
# -------------------------------------------------------
def resize_with_padding(img, target=TARGET_SIZE):
    h, w = img.shape[:2]
    aspect = w / h

    if aspect > 1:  # wide
        new_w = target
        new_h = int(target / aspect)
    else:  # tall
        new_h = target
        new_w = int(target * aspect)

    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # Add black padding
    top = (target - new_h) // 2
    bottom = target - new_h - top
    left = (target - new_w) // 2
    right = target - new_w - left

    padded = cv2.copyMakeBorder(
        resized, top, bottom, left, right,
        cv2.BORDER_CONSTANT, value=(0, 0, 0)
    )
    return padded


# -------------------------------------------------------
# Load Keras Model
# -------------------------------------------------------
print("Loading Keras model...")
model = load_model(MODEL_PATH)
print("Model loaded!")


# -------------------------------------------------------
# Initialize MediaPipe Hands
# -------------------------------------------------------
mp_draw = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

cap = cv2.VideoCapture(0)


# -------------------------------------------------------
# Main loop
# -------------------------------------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:

            # Extract bounding box
            xs = [lm.x for lm in hand_landmarks.landmark]
            ys = [lm.y for lm in hand_landmarks.landmark]

            xmin = int(min(xs) * w)
            xmax = int(max(xs) * w)
            ymin = int(min(ys) * h)
            ymax = int(max(ys) * h)

            pad = 30
            xmin = max(0, xmin - pad)
            ymin = max(0, ymin - pad)
            xmax = min(w, xmax + pad)
            ymax = min(h, ymax + pad)

            # Draw box
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

            # Crop
            crop = frame[ymin:ymax, xmin:xmax]

            if crop.size > 0:
                crop_96 = resize_with_padding(crop)

                # Show resized hand
                cv2.imshow("Hand 224x224", crop_96)

                # Prepare input for model
                inp = cv2.cvtColor(crop_96, cv2.COLOR_BGR2RGB)
                inp = inp.astype(np.float32) / 255.0
                inp = np.expand_dims(inp, axis=0)

                # Predict
                prediction = model.predict(inp)[0]

                # Get predicted class
                cls = np.argmax(prediction)
                conf = prediction[cls]

                # Display prediction
                text = f"Class: {class_names[cls]}  ({conf:.2f})"
                cv2.putText(frame, text, (xmin, ymin - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                # Draw landmarks
                mp_draw.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS
                )
            

    cv2.imshow("Live", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Loading Keras model...
Model loaded!


I0000 00:00:1765200408.626861   14979 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1765200408.632489   19686 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.1.5-1pop0~1756399231~22.04~b84bab8), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
W0000 00:00:1765200408.679643   19677 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765200408.713224   19683 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 921ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━